# An Automated Method for the Estimate of Vertical Jump Power through Intertial Measurement Units

This repository has been created as supplementary material/code source for the proceeding entitled *An Automated Method for the Estimate of Vertical Jump Power through Intertial Measurement Units* published for the $39^{\rm{th}}$ Conference of the International Society of Biomechanics in Sports (ISBS 2021). 

The aim of the study was to correct for IMU inaccuracies when trying to estimate concentric peak and mean power generated by a jumper during a vertical jump. The idea was to devise multiple linear regression models (MLRs) embedding both anthropometric and time-frequency parameters. 

In this Python Notebook you can find a brief example of the procedure applied to achieve a better power estimate by exploiting the MLR. The example provided here will take into account a sample Countermovement Jump, hence the following two MLRs apply:


|-| Constant ($b_0$) | $P_C$ ($b_1$) | $\overline{P}_C$ ($b_2$) | $P_E$ ($b_3$) | $\overline{P}_E$ ($b_4$) | B ($b_5$) | T ($b_6$) | S ($b_7$) | I  ($b_8$)| $P_\Delta$ ($b_9$) | $f_1$ ($b_{10}$) |
|-| ---------------- | ------------- | -------------------------| ------------- | ------------------------ | --------- | --------- | --------- | --------- | ------------------ | ---------------- |
| $MLR_{CMJ}^{P_C}$  |  10.40        | .47                      | .61           | -                        | -         |   -.24    | -         | -         | -.09               |    -     | -     | 
| $MLR_{CMJ}^{\overline{P}_C}$   |  9.2      | .16              | .2            | .14                      | 1.33         |   -.1    | -.08    | -.11         | -               |    -8.91     | -     | 


# Other resources
Notice that most of the signal processing have been alreadt made. To check how acceleration traces were processed (i.e. aligned with the world reference frame), please take a look to the other GitHub repository I made for a similar problem: **<a href = "https://github.com/Maskul93/MLR_ISB2021"> Smartphone-Based Democratization of Jump Height Estimate </a>**.

In [3]:
## -- Load packages -- ##
import numpy as np                     # Array package
import pandas as pd                    # Data-Frame package
import math                            # "Basic Advanced" maths operations
import matplotlib.pyplot as plt        # Advanced plot tool
from scipy import integrate, signal    # Signal processing tool